In [1]:
%run init.ipynb

In [2]:
sr = getTimeSeries();
sr

01.03.2018                         Name  Age          Nation...
04.09.2017                         Name  Age          Nation...
02.03.2017                         Name  Age          Nation...
01.09.2016                         Name  Age          Nation...
03.03.2016                         Name  Age          Nation...
01.09.2015                             Name  Age          Na...
06.03.2015                         Name  Age          Nation...
05.09.2014                         Name  Age          Nation...
dtype: object

Calculating team balance. Team receives player's value for every player that moved to another team(not retired - still exists in the database), and team loses player's value, if they bought him from other club(it was previously in the database).

In [3]:
clubs = set.intersection(*(set(data['Club'].unique()) for data in sr))
clubBalance = pd.Series(0, index=clubs);

for newData, oldData in reversed([*zip(sr, sr[1:])]):
    oldClubs = oldData.groupby('Club').apply(lambda players: set(players.index));
    newClubs = newData.groupby('Club').apply(lambda players: set(players.index));
    startingPlayers = set(newData.index) - set(oldData.index);
    retiredPlayers = set(oldData.index) - set(newData.index);
    for club in clubs:
        playersBought = newClubs[club] - oldClubs[club] - startingPlayers;
        playersSold = oldClubs[club] - newClubs[club] - retiredPlayers;
        clubBalance[club] -= sum([newData.loc[player, 'Value'] for player in playersBought]);
        clubBalance[club] += sum([newData.loc[player, 'Value'] for player in playersSold]);
clubProfits = clubBalance.sort_values(ascending=False);
pd.DataFrame(clubProfits).reset_index()

,index,0
0,AS Monaco,153475000
1,SL Benfica,121050000
2,LOSC Lille,108145000
3,Ajax,107965000
4,Fiorentina,105180000
5,Olympique Lyonnais,96680000
6,Borussia Dortmund,94135000
7,1. FSV Mainz 05,92950000
8,Empoli,91965000
9,Real Madrid CF,88160000


As this kind of data is not public, we can only compare it, for example to [this](https://football24.ua/shahtar_n467664/) urkainian article([google translated](https://translate.google.com.ua/translate?sl=uk&tl=en&js=y&prev=_t&hl=uk&ie=UTF-8&u=https%3A%2F%2Ffootball24.ua%2Fshahtar_n467664%2F&edit-text=&act=url)). First two clubs are the same. Others are in the similar spots. This article describes recent changes, that are not counted int this analisys.

Amounts are almost half of suggested bu the article, because, price of the players is set ususaly higher than his value.

As we can see, popular clubs are mostly in the bottom, because, they spen a lot of money on the best players. Joining this analysis with an average team ratings, we can determine clubs, which bring up good players to be succesfull, not just buy them.

In [4]:
clubMean = df.groupby('Club')['Overall'].mean();
clubMean = clubMean[clubs];
normMean = (clubMean - clubMean.mean()) / clubMean.std();
normMean = normMean - normMean.min() + 1e-5;
normBalance = (clubBalance - clubBalance.mean()) / clubBalance.std();
normBalance = normBalance - normBalance.min() + 1e-5;
clubProfitOverall = normMean * normBalance;
clubProfitOverall.sort_values(ascending=False)

Club
Real Madrid CF             48.872432
SL Benfica                 41.898983
AS Monaco                  40.918965
Sevilla FC                 39.954105
Borussia Dortmund          39.110887
Ajax                       35.462815
FC Barcelona               34.639010
Roma                       33.416083
Sporting CP                33.156550
Shakhtar Donetsk           32.227639
1. FSV Mainz 05            31.434986
FC Schalke 04              31.118968
Fiorentina                 30.688327
Bayer 04 Leverkusen        30.186721
RC Celta de Vigo           29.483523
SC Braga                   29.428950
Athletic Club de Bilbao    29.420426
Olympique Lyonnais         29.174266
Napoli                     29.126981
PSV                        28.756098
Real Sociedad              28.545686
Southampton                28.327746
Torino                     27.984235
Liverpool                  27.829093
TSG 1899 Hoffenheim        27.801805
RCD Espanyol               27.701809
Atalanta                   27.261